In [1]:
import pandas as pd 
data = pd.read_excel('IVM6311_Testing_scripts.xlsx', sheet_name='Trimming')

In [2]:
data.head()

,Parameter,Unnamed: 1,VBGR_ADJ_TRIM,TSDN,FRO_CLOCK,BST_OCP_TRIM
0,NaN,NaN,VERIFIED/NOT VERIFIED,VERIFIED/NOT VERIFIED,VERIFIED/NOT VERIFIED,VERIFIED/NOT VERIFIED
1,SL.NO,NaN,1,4,5,16
2,Priority,NaN,VBGR_ADJ_TRIM,TSDN,FRO_CLOCK,BST_OCP_TRIM
3,NaN,NaN,Run__Startup\nRun__Enable_Ana_Testpoint\n\n0xF...,Run__Startup\nRun__Enable_Ana_Testpoint\n\n0xF...,Run__Startup\n0xFE_0x00\n0x00_0x0D\n0xFE_0x01 ...,Run__startup\nRun__Boost_test_default\n\nWait_...
4,Final Test Limit,Min,1793,NaN,0xFE_0x01,NaN


In [3]:
from dft_syntaxparser import Parser
parser = Parser()

In [4]:
procedures = pd.read_excel('IVM6311_Testing_scripts.xlsx', sheet_name='Procedure')

In [5]:
procedures.columns

Index(['Startup', 'Enable_Ana_Testpoint', 'DACPA_tourn_on_wo_calibration',
       'Procedura_Maurizio', 'Boost_Test_Default', 'DAC-PA TDM_768_IN',
       'DAC-PA TDM_768_IN_and_Path_generator', 'DAC-PA PDM_3MHz_IN',
       'DAC-PA PDM_6MHz_IN', 'DAC-PA PDM_12MHz_IN',
       'DAC-PA ANALOG_IN_DC_COUPLED', 'DAC-PA ANALOG_IN_AC_COUPLED',
       'BOOST_FORCE_VBSO19V'],
      dtype='object')

In [123]:
from SwitchMatrix.mcp2221 import MCP2221
from SwitchMatrix.mcp2317 import MCP2317
mcp = MCP2221()
mcp2317 = MCP2317(mcp=mcp)
slave_address = 0x6c
def write_device(data:{}):
    device_data = mcp.mcpRead(SlaveAddress=slave_address,data=[int(data.get('RegAddr'),16)])[0]
    # print(f'Read data {hex(device_data)} ')
    bit_width = 2**(data.get('MSB') - data.get('LSB')+1)
    # print(bit_width)
    if int(data.get('Data'),16) < bit_width :
        mask = ~((bit_width-1) << data.get('LSB'))
        # print(f'mask {hex(256+mask)}')
        device_data = (device_data & mask) | ((int(data.get('Data'),16)) << data.get('LSB'))
        # print(f'Data write {hex(device_data)}')
        mcp.mcpWrite(SlaveAddress=slave_address, data=[int(data.get('RegAddr'),16),device_data])
        # written_data = mcp.mcpRead(SlaveAddress=slave_address,data=[int(data.get('RegAddr'),16)])[0]
        # print(f' data written {hex(written_data)}')
    else:
        print(f'Data is outof width ')
write_device({'RegAddr': '0x00', 'MSB': 7, 'LSB': 0, 'Data': '0xF'})

In [7]:
import re 
def execute_startup():
    startup_procedure = procedures['Startup'].loc[0].split('\n')
    for instruction in startup_procedure:
        instruction = instruction.lower()
        if re.match('0x',instruction):
            reg_data = parser.extract_RegisterAddress__Instruction(instruction) 
            # print(reg_data)
            write_device(reg_data)
        if re.match('Force__SDWN__1.8V'.lower(), instruction):
            print('Force 1.8V on SDWN')
            mcp2317.Switch(device_addr=0x23,row=8, col=4, Enable=True)
            
execute_startup()

Force 1.8V on SDWN
 data written 0x0
 data written 0xf
 data written 0x1
 data written 0x0
 data written 0x1


In [8]:
def execute_Enable_Ana_Testpoint():
    startup_procedure = procedures['Enable_Ana_Testpoint'].loc[0].split('\n')
    for instruction in startup_procedure:
        instruction = instruction.lower()
        if re.match('0x',instruction):
            print(parser.extract_RegisterAddress__Instruction(instruction)) 
            reg_data = parser.extract_RegisterAddress__Instruction(instruction) 
            write_device(reg_data)
        if re.match('FORCE__SDWN__OPEN'.lower(), instruction):
            print('Force SDWN OPEN')
            mcp2317.Switch(device_addr=0x23,row=8, col=4, Enable=False) # remove the 1.8V 

execute_Enable_Ana_Testpoint()

{'RegAddr': '0xfe', 'MSB': 7, 'LSB': 0, 'Data': '0x1'}
 data written 0x1
{'RegAddr': '0xf', 'MSB': 7, 'LSB': 0, 'Data': '0x88'}
 data written 0x88
{'RegAddr': '0x10', 'MSB': 7, 'LSB': 0, 'Data': '0x8'}
 data written 0x8
Force SDWN OPEN
{'RegAddr': '0x19', 'MSB': 7, 'LSB': 0, 'Data': '0x81'}
 data written 0x81
{'RegAddr': '0x1a', 'MSB': 7, 'LSB': 0, 'Data': '0x0'}
 data written 0x0


In [88]:
import re 
def typical_value_clean(value:str):
    value = (lambda value : value.replace(',','.') if re.findall(',',value) else value)(value=value)
    value = (lambda value : float(value.replace('m',''))*10**-3  if isinstance(value,str)    and re.findall('m',value) else value)(value=value)
    value = (lambda value : float(value.replace('n',''))*10**-9  if isinstance(value,str)    and re.findall('n',value)  else value)(value=value)
    value = (lambda value : float(value.replace('u',''))*10**-6  if isinstance(value,str)    and re.findall('u',value)  else value)(value=value)
    value = (lambda value : float(value.replace('k',''))*10**3   if isinstance(value,str)    and re.findall('k',value)  else value)(value=value)
    value = (lambda value : float(value.replace('M',''))*10**6   if isinstance(value,str)    and re.findall('M',value)  else value)(value=value)
    value = (lambda value : float(value.replace('G',''))*10**9   if isinstance(value,str)    and re.findall('G',value)  else value)(value=value)
    if not isinstance(value,float) :
        value = float(value)
    return value
typical_value_clean('1,2')

1.2

In [9]:
import re 
from time import sleep
from Instruments.Keysight_34461 import A34461
from Instruments.DigitalScope import dpo_2014B
def calculate__minerror(measure_signal:{},trim_reg:dict,typical:float):
    if measure_signal and trim_reg:
        bit_width = trim_reg.get('MSB') - trim_reg.get('LSB')+1
        signal_Unit = measure_signal.get('Unit')
        measure_values = []
        code = []
        print(signal_Unit)
        if re.search('voltage', signal_Unit):
            meter = A34461('USB0::0x2A8D::0x1401::MY57216238::INSTR')
            mcp2317.Switch(device_addr=0x23,row=8, col=1, Enable=True) # Enable the Volt meter 
            print(f'>>>>>>>>>>>>>> {2**bit_width}')
            for i in range(2**bit_width):
                trim_reg.update({
                    'Data': hex(i)
                })
                code.append(i)
                print(trim_reg)
                write_device(trim_reg)
                sleep(0.1)
                measure_values.append(meter.meas_V())
                print(f'code {code[-1]} value : {measure_values[-1]}')
        if re.search('frequency', signal_Unit):
            scope = dpo_2014B('USB0::0x0699::0x0456::C014545::INSTR')
            typical = 13*(10**6)
            scope.set_trigger__mode(mode='NORM')
            scope.set_HScale()
            scope.set_Channel__VScale(scale=0.5)

            mcp2317.Switch(device_addr=0x27,row=7, col=3, Enable=True) # Enable the DSO 
            input(f'>>>>>>>>>>>>>> {2**bit_width}')
            for i in range(2**bit_width):
                trim_reg.update({
                    'Data': hex(i)
                })
                code.append(i)
                print(trim_reg)
                write_device(trim_reg)
                sleep(0.1)
                freq=0
                for i in range(0,20):
                    freq= freq + scope.meas_Freq()
                measure_values.append(freq)
                print(f'code {code[-1]} value : {measure_values[-1]}')
        # if re.findall('curr', signal_Unit):
        #     meter = Curremeter()
        if code and measure_values:
            min_val = min(measure_values, key=lambda x: abs(x-typical))
            index = measure_values.index(min_val)
            optimal_code = code[index]
            print(f'optimal value {measure_values[index]} optimal code {optimal_code}')

calculate__minerror({},{},0.0)


In [85]:
import re 
def trimming_DFT(data=pd.DataFrame({}),test_name=''):
    test_name = test_name
    instructions = data[test_name].loc[3].split('\n')
    typical = typical_value_clean(data[test_name].loc[6]) 
    for instruction in instructions:
        instruction = instruction.lower()
        # parse the instructions
        if re.match('run',instruction):
            if re.findall('startup', instruction):
               print('Startup Procedure')
               execute_startup()
            if re.findall('enable_ana_testpoint', instruction):
                print('execute_Enable_Ana_Testpoint Procedure')
                execute_Enable_Ana_Testpoint()
        if re.match('0x',instruction):
            reg_data = parser.extract_RegisterAddress__Instruction(instruction)
            print(reg_data)
            write_device(reg_data)
        if re.match('measure',instruction):
            measure_signal = parser.extract_Measure__Instruction(instruction)
            print(f'measure_signal  > {measure_signal}')
        if re.match('trim',instruction):
            print(instruction)
            trim_reg = parser.extract_TrimSweep__Instruction(instruction) 
            print(trim_reg)
        if re.match('calculate__minerror',instruction):
            print('execute min error function')
            calculate__minerror(measure_signal=measure_signal, trim_reg=trim_reg, typical=typical)

## Read Tests yaml file 


In [84]:
from box import ConfigBox
from box.exceptions import BoxValueError
import os 
import yaml
from pathlib import Path

def read_yaml(path_to_yaml: Path) -> ConfigBox:
    """reads yaml file and returns

    Args:
        path_to_yaml (str): path like input

    Raises:
        ValueError: if yaml file is empty
        e: empty file

    Returns:
        ConfigBox: ConfigBox type
    """
    try:
        with open(path_to_yaml) as yaml_file:
            content = yaml.safe_load(yaml_file)
            return ConfigBox(content)
    except BoxValueError:
        raise ValueError("yaml file is empty")
    except Exception as e:
        raise e

def DFT_Tests(path='Tests.yaml'):
    return read_yaml(path_to_yaml=path)

In [96]:
tests = DFT_Tests()
print(tests.Trim)

['VBGR_ADJ_TRIM', 'TSDN']


In [146]:
trimming_data = pd.read_excel('IVM6311_Testing_scripts.xlsx', sheet_name='Trimming')
for test in tests.Trim:
    print(f'....... {test}')
    trimming_DFT(trimming_data, test)

....... VBGR_ADJ_TRIM
Startup Procedure
Force 1.8V on SDWN
execute_Enable_Ana_Testpoint Procedure
{'RegAddr': '0xfe', 'MSB': 7, 'LSB': 0, 'Data': '0x1'}
{'RegAddr': '0xf', 'MSB': 7, 'LSB': 0, 'Data': '0x88'}
{'RegAddr': '0x10', 'MSB': 7, 'LSB': 0, 'Data': '0x8'}
Force SDWN OPEN
{'RegAddr': '0x19', 'MSB': 7, 'LSB': 0, 'Data': '0x81'}
{'RegAddr': '0x1a', 'MSB': 7, 'LSB': 0, 'Data': '0x0'}
{'RegAddr': '0xfe', 'MSB': 7, 'LSB': 0, 'Data': '0x1'}
{'RegAddr': '0x1a', 'MSB': 7, 'LSB': 0, 'Data': '0x1'}
{'RegAddr': '0xb0', 'MSB': 7, 'LSB': 0, 'Data': '0xe'}
measure_signal  > {'Signal': 'sdwn', 'Unit': 'voltage'}
trimsweep - 0xb0[7:4] "select code which sets atest voltage as close as possible to target"
{'RegAddr': '0xb0', 'MSB': 7, 'LSB': 4, 'Data': None}
execute min error function
voltage
>>>>>>>>>>>>>> 16
{'RegAddr': '0xb0', 'MSB': 7, 'LSB': 4, 'Data': '0x0'}
code 0 value : 1.8215435875
{'RegAddr': '0xb0', 'MSB': 7, 'LSB': 4, 'Data': '0x1'}
code 1 value : 1.81542344
{'RegAddr': '0xb0', 'MSB':

In [133]:
import re 
from time import sleep
from Instruments.Keysight_34461 import A34461
from Instruments.DigitalScope import dpo_2014B
from Instruments.KeySight_N670x import N670x
def measure_value_check(measure_signal:{},typical:float):
    if measure_signal :
        signal_Unit = measure_signal.get('Unit')
        measure_values = None
        print(signal_Unit)
        if re.search('voltage', signal_Unit):
            meter = A34461('USB0::0x2A8D::0x1401::MY57216238::INSTR')
            mcp2317.Switch(device_addr=0x23,row=8, col=1, Enable=True) # Enable the Volt meter 
            sleep(0.1)
            measure_values = meter.meas_V()
            print(f' value : {measure_values}')

        if re.search('current', signal_Unit):
            meter = N670x('USB0::0x0957::0x0F07::MY50002157::INSTR')
            mcp2317.Switch(device_addr=0x23,row=8, col=5, Enable=True) # Enable the Volt meter 
            sleep(0.1)
            meter.outp_ON(channel=3)
            sleep(1)
            measure_values = meter.getCurrent(channel=3)
            print(f' value : {measure_values}')
            sleep(1)
            meter.outp_OFF(channel=3)

measure_value_check({},0.0)

In [128]:
def reference_DFT(data=pd.DataFrame({}),test_name=''):
    test_name = test_name
    instructions = data[test_name].loc[3].split('\n')
    typical = typical_value_clean(data[test_name].loc[6]) 
    for instruction in instructions:
        instruction = instruction.lower()
        print(instruction)
        # parse the instructions
        if re.match('run',instruction):
            if re.findall('startup', instruction):
               print('Startup Procedure')
               execute_startup()
            if re.findall('enable_ana_testpoint', instruction):
                print('execute_Enable_Ana_Testpoint Procedure')
                execute_Enable_Ana_Testpoint()
        if re.match('0x',instruction):
            reg_data = parser.extract_RegisterAddress__Instruction(instruction)
            print(reg_data)
            write_device(reg_data)
        if re.match('measure',instruction):
            print(instruction)
            measure_signal = parser.extract_Measure__Instruction(instruction)
            print(f'measure_signal  > {measure_signal}')
            measure_value_check(measure_signal=measure_signal,typical=typical)

In [147]:
reference_data = pd.read_excel('IVM6311_Testing_scripts.xlsx', sheet_name='Reference')
tests = DFT_Tests()
for test in tests.Reference:
    print(f'............ {test}')
    reference_DFT(reference_data, test)


............ UVLO
force__vbat__3.6v
force__sdwn__1.8v

0xfe_0x01 "select page 1"
{'RegAddr': '0xfe', 'MSB': 7, 'LSB': 0, 'Data': '0x1'}
0x2f_0xaa "unlock test page"
{'RegAddr': '0x2f', 'MSB': 7, 'LSB': 0, 'Data': '0xaa'}

run__enable_ana_testpoint
execute_Enable_Ana_Testpoint Procedure
{'RegAddr': '0xfe', 'MSB': 7, 'LSB': 0, 'Data': '0x1'}
{'RegAddr': '0xf', 'MSB': 7, 'LSB': 0, 'Data': '0x88'}
{'RegAddr': '0x10', 'MSB': 7, 'LSB': 0, 'Data': '0x8'}
Force SDWN OPEN
{'RegAddr': '0x19', 'MSB': 7, 'LSB': 0, 'Data': '0x81'}
{'RegAddr': '0x1a', 'MSB': 7, 'LSB': 0, 'Data': '0x0'}

0x1a_0x00 "uvlo"
{'RegAddr': '0x1a', 'MSB': 7, 'LSB': 0, 'Data': '0x0'}

measure__voltage__sdwn
measure__voltage__sdwn
measure_signal  > {'Signal': 'sdwn', 'Unit': 'voltage'}
voltage
 value : 3.9995397400000003

0x2f_0xbb "lock test page"
{'RegAddr': '0x2f', 'MSB': 7, 'LSB': 0, 'Data': '0xbb'}
0xfe_0x00 "return page 0"
{'RegAddr': '0xfe', 'MSB': 7, 'LSB': 0, 'Data': '0x0'}
............ OUT_TEST_FRO_VVCO
run__startup 